In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import gzip
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# data path
path_csi =  'J:\\Data\\Wi-Fi_processed\\'
path_csi_hc = 'J:\\Data\\Wi-Fi_HC\\180_100\\'

# data info
df_info = pd.read_csv('data_subc_sig_v1.csv')
# parameters
max_value = np.max(df_info['max'].values)
no_classes = np.max(df_info['id_person'])+1
epochs = 100
batch_size = 10
csi_time = int(np.max(df_info['len']))
csi_subc = 30
input_shape = (csi_time, csi_subc, 6)

C:\Users\herok\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# make data generator
def gen_csi(df_info,id_num,len_num):
    for file in set(df_info.id.values):
        # read sample data
        # load and uncompress.
        with gzip.open(path_csi+file+'.pickle.gz','rb') as f:
            data1 = pickle.load(f)
        data1_abs = np.abs(data1) / max_value
        # zero pad
        pad_len = len_num - data1_abs.shape[0]
        data1_pad = np.pad(data1_abs,((0,pad_len),(0,0),(0,0),(0,0)),'constant',constant_values=0)

        data1_xs = data1_pad.astype('float32')    
        data1_y = df_info[df_info.id==file]['id_person'].values[0].astype('int')

        x_input = data1_xs.reshape([-1,len_num,30,6])
        y_input = tf.keras.utils.to_categorical(data1_y, no_classes).reshape([1,-1])

        #xs_input.append(x_input)
        #ys_input.append(y_input)
        
        yield(x_input,y_input)

In [3]:
generator = gen_csi(df_info,csi_subc,csi_time)
a,b = next(generator)

In [4]:
#models
def simple_cnn(input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        activation='relu',
        input_shape=input_shape
    ))
    model.add(tf.keras.layers.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        activation='relu'
    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=10))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=256, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(tf.keras.layers.Dense(units=no_classes, activation='softmax'))
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

In [ ]:
simple_cnn_model = simple_cnn(input_shape)
gen = gen_csi(df_info,no_classes,csi_time)
simple_cnn_model.fit_generator(gen,steps_per_epoch=10,epochs=100)

Epoch 1/100
10/10 [==============================] - 15s 2s/step - loss: 14.9630 - acc: 0.0000e+00
Epoch 2/100
10/10 [==============================] - 2s 208ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 3/100
10/10 [==============================] - 3s 336ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 4/100
10/10 [==============================] - 4s 364ms/step - loss: 14.5063 - acc: 0.1000
Epoch 5/100
10/10 [==============================] - 3s 328ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 6/100
10/10 [==============================] - 3s 339ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 7/100
10/10 [==============================] - 3s 339ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 8/100
10/10 [==============================] - 3s 331ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 9/100
10/10 [==============================] - 4s 361ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 10/100
10/10 [==============================] - 3s 345ms/step - loss: 16.1181 - acc: 0.0000

In [ ]:
a,b = next(gen)

In [ ]:
len(set(df_info.id.values))